In [98]:
!pip install langchain
!pip install openai
!pip install tiktoken
!pip install pypdf
!pip install -U langchain-community

In [99]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import os

In [100]:
mkdir pdfs

A subdirectory or file pdfs already exists.


In [101]:
loader = PyPDFDirectoryLoader('pdfs')


In [102]:


data=loader.load()

In [103]:
data[0]

Document(metadata={'producer': 'iLovePDF', 'creator': 'PyPDF', 'creationdate': '', 'moddate': '2025-08-24T16:13:42+00:00', 'source': 'pdfs\\DA1.pdf', 'total_pages': 123, 'page': 0, 'page_label': '1'}, page_content='ADTA 5130/IPAC 4130\nMODULE 1:  CHAPTERS 1, 2, AND 3')

In [104]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=20)                     

In [105]:
text_chunks = text_splitter.split_documents(data)

In [106]:
text_chunks

[Document(metadata={'producer': 'iLovePDF', 'creator': 'PyPDF', 'creationdate': '', 'moddate': '2025-08-24T16:13:42+00:00', 'source': 'pdfs\\DA1.pdf', 'total_pages': 123, 'page': 0, 'page_label': '1'}, page_content='ADTA 5130/IPAC 4130\nMODULE 1:  CHAPTERS 1, 2, AND 3'),
 Document(metadata={'producer': 'iLovePDF', 'creator': 'PyPDF', 'creationdate': '', 'moddate': '2025-08-24T16:13:42+00:00', 'source': 'pdfs\\DA1.pdf', 'total_pages': 123, 'page': 1, 'page_label': '2'}, page_content='Agenda\n• Introduction\n• Admin: Canvas, Syllabus, Expectations\n• Chapters 1, 2, and 3\n• Homework'),
 Document(metadata={'producer': 'iLovePDF', 'creator': 'PyPDF', 'creationdate': '', 'moddate': '2025-08-24T16:13:42+00:00', 'source': 'pdfs\\DA1.pdf', 'total_pages': 123, 'page': 2, 'page_label': '3'}, page_content='Definitions\n• Descriptive stats:  characteristics of data; charts/tables/measures\n• Inferential stats:  drawing conclusions about data'),
 Document(metadata={'producer': 'iLovePDF', 'creator'

In [107]:
print(text_chunks[6].page_content)

Visualizing Categorical Variables
Bar Chart Pie Chart Stacked Bar Chart
One Variable Two Variables


In [108]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [109]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


os.environ["OPENAI_API_KEY"]= OPENAI_API_KEY

In [110]:
embedding = OpenAIEmbeddings()

In [111]:
len(embedding.embed_query(text_chunks[6].page_content))

1536

In [112]:
len(embedding.embed_query("How are you?"))

1536

In [113]:
from pinecone import Pinecone, ServerlessSpec 
#PINECONE_API_KEY= os.environ.get("PINECONE_API_KEY","pcsk_62zXe8_NsmcMGGcY57J4KNEYysAgXPFSEXyC9U6qUZukfAVbBoKVy8s55rMpTVZfHgxyRu")
PINECONE_API_KEY="pcsk_62zXe8_NsmcMGGcY57J4KNEYysAgXPFSEXyC9U6qUZukfAVbBoKVy8s55rMpTVZfHgxyRu"
os.environ["PINECONE_API_KEY"]=PINECONE_API_KEY

In [114]:
from pinecone import Pinecone, ServerlessSpec   # Pinecone SDK
from langchain_pinecone import PineconeVectorStore  # LangChain wrapper


In [115]:
pc = Pinecone(api_key=PINECONE_API_KEY)

In [116]:
index_name = "testing"

In [117]:
if index_name not in [i["name"] for i in pc.list_indexes()]:
    pc.create_index(
        name=index_name,
        dimension=1536,        # Must match your embedding model dimension
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

In [118]:
index = pc.Index(index_name)

# Create Embeddings for each of the Text Chunk

In [119]:
docsearch = PineconeVectorStore.from_texts(
    texts=[t.page_content for t in text_chunks],
    embedding=embedding,
    index_name=index_name
)

In [120]:
docsearch

In [121]:
docsearch.as_retriever()

VectorStoreRetriever(tags=['PineconeVectorStore', 'OpenAIEmbeddings'], vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x000001EACA045F90>, search_kwargs={})

In [122]:
query="machine learning models"

In [123]:
docs=docsearch.similarity_search(query)

In [124]:
docs

[Document(id='d76ef2af-aa2c-40ef-8182-6a960aff8c53', metadata={}, page_content='machine learning is not built on a pre-structured model; rather, the data shape the \nmodel by detecting underlying patterns. The more variables (input) used to train the \nmodel, the more accurate the ultimate model will be.[35] \nLeo Breiman distinguished two statistical modelling paradigms: data model and \nalgorithmic model,[36] wherein "algorithmic model" means more or less the machine \nlearning algorithms like Random Forest.'),
 Document(id='87eb9573-d235-440a-935b-42a1e053bcbd', metadata={}, page_content='machine learning is not built on a pre-structured model; rather, the data shape the \nmodel by detecting underlying patterns. The more variables (input) used to train the \nmodel, the more accurate the ultimate model will be.[35] \nLeo Breiman distinguished two statistical modelling paradigms: data model and \nalgorithmic model,[36] wherein "algorithmic model" means more or less the machine \nlearn

In [125]:
llm = OpenAI()

In [126]:
qa= RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())

In [127]:
query="use of machine learning models"

In [128]:
qa.run(query)

' The use of machine learning models is to help develop and make predictions for future outcomes based on data. It can be applied to various fields such as healthcare and finance, and is often related to artificial intelligence. '

In [130]:
 
import sys
while True:
    user_input = input("Enter your question (or 'exit' to quit):" )
    if user_input.lower() == 'exit':
        print("Exiting the program.")
        sys.exit()  
    if user_input=="":
        continue
    result=qa({"query":user_input})
    print(f"Answer: {result['result']}")
        
    

Answer:  Descriptive and Inferential statistics.
Answer: 
Categorical and discrete variables are not continuous.
Answer:  I don't know.
Exiting the program.


SystemExit: 

c:\Users\Narasimha\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
